In [1]:
import numpy as np
import pandas as pd
import healpy as hp
import GCRCatalogs

In [2]:
cat = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')
cosmodc2_nside = 32

In [3]:
# change these quantities as needed
your_data = [(9687500000, 62.58, -40.33), (10500000002, 62.32, -41.86)]
quantities_needed = ['galaxy_id', 'ra', 'dec', 'mag_r']

In [4]:
host_galaxy_props = []

for host_id, sn_ra, sn_dec in your_data:
    pixels = [hp.ang2pix(cosmodc2_nside, sn_ra, sn_dec, lonlat=True)]
    pixels.extend(hp.get_all_neighbours(cosmodc2_nside, sn_ra, sn_dec, lonlat=True))
    for pixel in pixels:
        # remove 'redshift_block_lower == 0' in `native_filters` below if expecting high-z hosts
        data = cat.get_quantities(
            quantities_needed, 
            filters='galaxy_id == {}'.format(host_id), 
            native_filters=['healpix_pixel == {}'.format(pixel), 'redshift_block_lower == 0']
        )
        if len(data[quantities_needed[0]]):
            host_galaxy_props.append(pd.DataFrame(data))
            break
    else:
        print('Cannot find host {} near ({}, {})'.format(host_id, sn_ra, sn_dec))

host_galaxy_props = pd.concat(host_galaxy_props, ignore_index=True)

In [5]:
host_galaxy_props

,mag_r,galaxy_id,ra,dec
0,15.969118,9687500000,62.584977,-40.334110
1,17.833668,10500000002,62.319568,-41.860047
